In [79]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

In [80]:
df = pd.read_excel("/home/Code/PhD/data_collection_system/file/testing_jieba.xlsx")

In [81]:
df['label2']

0      2
1      1
2      1
3      1
4      1
      ..
907    2
908    2
909    2
910    2
911    0
Name: label2, Length: 912, dtype: int64

In [82]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [83]:
#part_of_speech_list = ["A","Caa","Cab","Cba","Cbb","D","Da","Dfa","Dfb","Di","Dk","DM","I","Na","Nb","Nc","Ncd","Nd","Nep","Neq","Neq","Nes","Neu","Nf","Ng","Nh","Nv","P","T","VA","VAC","VB","VC","VCL","VD","VF","VE","VG","VH","VHC","VI","VJ","VK","VL","V_2","DE","SHI","FW"]

In [84]:
part_of_speech_list = [
    "n","f","s","t","nr","ns","nt","nw",
    "nz","v","vd","vn","a","ad","an",
    "d","m","q","r","p","c","u","xc",
    "w","PER","LOC","ORG","TIME"
]

In [85]:
text_enum = {}
for index, word in enumerate(part_of_speech_list):
    text_enum[word] = index

In [86]:
df_orig = df 

In [87]:
for item in text_enum.items():
    df = df.replace(item[0], item[1])

In [88]:
df_col = list(df.columns)

In [89]:
df_col[list(df.columns).index("<script async")] = df_col[list(df.columns).index("<script async")].replace("<","")

In [90]:
df.columns = df_col

In [91]:
drop_index = df[df['label2'] == 2].index

In [92]:
drop_index

Int64Index([  0,   7,   8,  10,  12,  19,  22,  23,  25,  26,
            ...
            899, 900, 901, 903, 904, 905, 907, 908, 909, 910],
           dtype='int64', length=581)

In [93]:
df.drop(drop_index , inplace = True)

In [94]:
df['label2']

1      1
2      1
3      1
4      1
5      1
      ..
887    1
896    1
902    1
906    0
911    0
Name: label2, Length: 331, dtype: int64

In [95]:
len(df[df['label2'] == 0]), len(df[df['label2'] == 1])

(32, 299)

In [96]:
#Using Pearson Correlation
cor = df.corr()
cor_target = abs(cor["label"])
#print(cor_target.sort_values(ascending = False))

In [97]:
import xgboost as xgb


In [98]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, random_state=777, train_size=0.75)
len(train_data),len(test_data)

(248, 83)

In [99]:
train_data_Y = train_data['label']
test_data_Y = test_data['label']

In [100]:
train_data = train_data.drop('label',axis= 1)
test_data = test_data.drop('label',axis= 1)

In [101]:
train_data.drop("url",axis = 1, inplace = True)
train_data.drop("company",axis = 1, inplace = True)

In [102]:
test_data.columns

Index(['most common number 1', 'most common number 2', 'most common number 3',
       'most common number 4', 'most common number 5', 'most common number 6',
       'most common number 7', 'most common number 8', 'most common number 9',
       'most common number 10', 'n', 'f', 's', 't', 'nr', 'ns', 'nt', 'nw',
       'nz', 'v', 'vd', 'vn', 'a', 'ad', 'an', 'd', 'm', 'q', 'r', 'p', 'c',
       'u', 'xc', 'w', 'PER', 'LOC', 'ORG', 'TIME', 'script async', 'id',
       'class', 'css', 'header', 'table', 'script', 'footer', 'img', 'src',
       'form', 'br', 'div', 'section', 'main', 'title', 'link', 'href',
       'company', 'url', 'label2'],
      dtype='object')

In [103]:
test_data_orig = test_data.copy()

In [104]:
test_data.drop("url",axis = 1, inplace = True)
test_data.drop("company",axis = 1, inplace = True)

In [105]:
X_res, y_res = train_data, train_data_Y

In [106]:
from collections import Counter


In [107]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=100, learning_rate=0.01,scale_pos_weight=int(Counter(y_res)[0]/Counter(y_res)[1])).fit(X_res, y_res)

In [108]:
test_y_predicted = gbm.predict(test_data)

from sklearn import  metrics
accuracy = metrics.accuracy_score(test_data_Y,test_y_predicted)
print(accuracy)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_data_Y,test_y_predicted))
a,b = confusion_matrix(test_data_Y,test_y_predicted)

print(b[1]/(b[0]+b[1]))

from sklearn.metrics import roc_auc_score
print("ROC :",roc_auc_score(test_data_Y,test_y_predicted))

0.9759036144578314
[[80  1]
 [ 1  1]]
0.5
ROC : 0.7438271604938271


In [109]:
for i in range(len(test_data_Y)):
    if list(test_data_Y)[i] != list(test_y_predicted)[i]:
        print(list(test_data_Y)[i],list(test_y_predicted)[i],test_data_orig.iloc[i]['url'], test_data_orig.iloc[i]['company'])

1 0 https://blog.csdn.net/quanquanxiaobu/article/details/106317195 Cadence
0 1 https://ptt.reviews/Tech_Job/M.1623165621.A.69C TSMC


In [29]:
importance_features = gbm.feature_importances_
features = {}
for i in range(len(importance_features)):
    features[X_res.columns[i]] = importance_features[i]
features = sorted(features.items(), key=lambda x: x[1], reverse=True)
features = pd.Series(features)
print(features)

0              (script async, 0.27260995)
1                       (main, 0.1417183)
2                    (script, 0.08994146)
3                        (p, 0.079265244)
4                      (link, 0.06244963)
5                       (id, 0.058888763)
6                      (div, 0.056756165)
7       (most common number 1, 0.0525044)
8                   (footer, 0.045867067)
9                       (br, 0.036563277)
10                      (css, 0.02978842)
11    (most common number 6, 0.024391156)
12    (most common number 5, 0.024344344)
13                  (header, 0.020216769)
14                  (title, 0.0046949796)
15            (most common number 2, 0.0)
16            (most common number 3, 0.0)
17            (most common number 4, 0.0)
18            (most common number 7, 0.0)
19            (most common number 8, 0.0)
20            (most common number 9, 0.0)
21           (most common number 10, 0.0)
22                               (n, 0.0)
23                               (

In [30]:
# save model
#import pickle 
#file_name = '../../model/xgboost_low_blood1_'+ str(round(accuracy,4)) + '.pickle'
#with open(file_name, 'wb') as f:
#    pickle.dump(gbm, f)